# Regresión lineal múltiple con Python y Scikit-Learn

## Importación de librerías

In [1]:
import gc
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error

## Lectura de datos y eliminación de columna innecesaria.

In [2]:
df_nir_train = pd.read_excel('MATRIZ FINAL.xlsx', sheet_name='calibracion')
df_nir_val = pd.read_excel('MATRIZ FINAL.xlsx', sheet_name='validacion')
df_nir_train.drop('Unnamed: 0', axis=1, inplace=True)
df_nir_val.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
print(df_nir_train.shape, df_nir_val.shape)

(266, 1109) (118, 1109)


## Separación de X's y Y's

In [4]:
X_train = df_nir_train.iloc[:,:1103]
X_test = df_nir_val.iloc[:,:1103]
Y_train = df_nir_train.iloc[:,1103:]
Y_test = df_nir_val.iloc[:,1103:]

## Normalización de Datos

In [5]:
standard_scaler = StandardScaler()
standard_scaler.fit(X_train)
X_train_scaled = standard_scaler.transform(X_train)
X_test_scaled = standard_scaler.transform(X_test)

standard_scaler.fit(Y_train)
Y_train_scaled = standard_scaler.transform(Y_train)
Y_test_scaled = standard_scaler.transform(Y_test)

In [8]:
Y_train_scaled.shape

(266, 6)

## Separación de las Y's en train y test

In [23]:
traccion = Y_train_scaled.T[0].reshape(-1,1)
energia = Y_train_scaled.T[1].reshape(-1,1)
tensil = Y_train_scaled.T[2].reshape(-1,1)
elongacion = Y_train_scaled.T[3].reshape(-1,1)
secante = Y_train_scaled.T[4].reshape(-1,1)
flexion = Y_train_scaled.T[5].reshape(-1,1)

In [31]:
traccion_test = Y_test_scaled.T[0].reshape(-1,1)
energia_test = Y_test_scaled.T[1].reshape(-1,1)
tensil_test = Y_test_scaled.T[2].reshape(-1,1)
elongacion_test = Y_test_scaled.T[3].reshape(-1,1)
secante_test = Y_test_scaled.T[4].reshape(-1,1)
flexion_test = Y_test_scaled.T[5].reshape(-1,1)

## Regresiones

### Tracción

In [32]:
regression = LinearRegression()
regression.fit(X_train_scaled, traccion)
print("Training")
print(regression.score(X_train_scaled, traccion))
y = regression.predict(X_train_scaled)
print(r2_score(traccion, y))
print(mean_squared_error(traccion, y))
print("Validation")
y_test = regression.predict(X_test_scaled)
print(regression.score(X_test_scaled, traccion_test))
print(r2_score(traccion_test, y_test))

Training
1.0
1.0
1.0389514344953171e-23
Validation
-2.9111823003912902
-2.9111823003912902


### Energia

In [34]:
regression = LinearRegression()
regression.fit(X_train, energia)
print("Training")
print(regression.score(X_train, energia))
y = regression.predict(X_train)
print(r2_score(energia, y))
print(mean_squared_error(energia, y))
print("Validation")
y_test = regression.predict(X_test)
print(regression.score(X_test, energia_test))
print(r2_score(energia_test, y_test))

Training
1.0
1.0
1.0310050947266875e-22
Validation
-1.4338529283553227
-1.4338529283553227


Esto tiene un claro aspecto de que está ocurriendo **overfitting**. Vamos a reducirlo con regularización. Probé varias para tracción y solo funciona Ridge solo, Lasso desmejora, ElasticNet desmejora, Cross-Validation desmejora a Ridge.

In [47]:
from sklearn.linear_model import RidgeCV

# Crear un modelo de regresión Ridge con validación cruzada
ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1.0, 3.0, 5.0, 7.0, 10.0], cv=5)

# Entrenar el modelo con los datos de entrenamiento escalados
ridge_cv.fit(X_train_scaled, traccion)

# Obtener el valor óptimo de alpha seleccionado
best_alpha = ridge_cv.alpha_
print("Mejor valor de alpha:", best_alpha)

# Evaluar el modelo en el conjunto de prueba escalado
score = ridge_cv.score(X_test_scaled, traccion_test)
print("Puntaje en el conjunto de entrenamiento:", ridge_cv.score(X_train_scaled, traccion))
print("Puntaje en el conjunto de prueba:", score)


Mejor valor de alpha: 1.0
Puntaje en el conjunto de entrenamiento: 0.9316874637489747
Puntaje en el conjunto de prueba: 0.9385134934380096


In [50]:
from sklearn.linear_model import LassoCV, ElasticNetCV

# Lasso
lasso_cv = LassoCV(alphas=[0.1, 1.0, 10.0], cv=5)
lasso_cv.fit(X_train_scaled, traccion)
lasso_alpha = lasso_cv.alpha_
print("Alfa de Lasso:", lasso_alpha)
lasso_score = lasso_cv.score(X_test_scaled, traccion_test)
print("Score de Lasso en training:", lasso_cv.score(X_train_scaled, traccion))
print("Score de Lasso en test:", lasso_score)

# ElasticNet
elasticnet_cv = ElasticNetCV(alphas=[0.1, 1.0, 10.0], l1_ratio=[0.5, 0.7, 0.9], cv=5)
elasticnet_cv.fit(X_train_scaled, traccion)
elasticnet_alpha = elasticnet_cv.alpha_
print("Alfa de ElasticNet:", elasticnet_alpha)
elasticnet_ratio = elasticnet_cv.l1_ratio_
elasticnet_score = elasticnet_cv.score(X_test_scaled, traccion_test)
print("Score de EL en train:", elasticnet_cv.score(X_train_scaled, traccion))
print("Score de EL en test:", elasticnet_score)

/home/robertbarac/maestria/seminario_en_ciencias/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Alfa de Lasso: 0.1
Score de Lasso en training: 0.28264351771385265
Score de Lasso en test: 0.28251729159333605


/home/robertbarac/maestria/seminario_en_ciencias/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Alfa de ElasticNet: 0.1
Score de EL en train: 0.5575052150172006
Score de EL en test: 0.5516220109916492


In [51]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# Crear un modelo de regresión Ridge
ridge = Ridge(alpha=0.5)

# Aplicar validación cruzada
scores = cross_val_score(ridge, X_train_scaled, traccion, cv=5, scoring='r2')

# Calcular el promedio de los puntajes de validación cruzada
mean_score = scores.mean()

print("Puntajes de validación cruzada:", scores)
print("Promedio del puntaje de validación cruzada:", mean_score)


Puntajes de validación cruzada: [ 0.54522883 -0.43877735  0.30994174  0.48138872 -1.04090759]
Promedio del puntaje de validación cruzada: -0.028625129111638437


### Energía

In [59]:
# Crear un modelo de regresión Ridge con validación cruzada
ridge_cv = RidgeCV(alphas=[30.0, 35.0, 36.0, 38.0, 39.0, 40.0], cv=5)

# Entrenar el modelo con los datos de entrenamiento escalados
ridge_cv.fit(X_train_scaled, energia)

# Obtener el valor óptimo de alpha seleccionado
best_alpha = ridge_cv.alpha_
print("Mejor valor de alpha:", best_alpha)

# Evaluar el modelo en el conjunto de prueba escalado
score = ridge_cv.score(X_test_scaled, energia_test)
print("Puntaje en el conjunto de entrenamiento:", ridge_cv.score(X_train_scaled, traccion))
print("Puntaje en el conjunto de prueba:", score)

Mejor valor de alpha: 35.0
Puntaje en el conjunto de entrenamiento: -1.2726301749218334
Puntaje en el conjunto de prueba: 0.7585780805534141


Vemos que no está funcionando del todo bien. Probemos con Lasso a ver qué tal

In [66]:
# Lasso
lasso_cv = LassoCV(alphas=[0.1, 1.0, 10.0], cv=5)
lasso_cv.fit(X_train_scaled, energia)
lasso_alpha = lasso_cv.alpha_
print("Alfa de Lasso:", lasso_alpha)
lasso_score = lasso_cv.score(X_test_scaled, energia_test)
print("Score de Lasso en training:", lasso_cv.score(X_train_scaled, energia))
print("Score de Lasso en test:", lasso_score)

# ElasticNet
elasticnet_cv = ElasticNetCV(alphas=[0.1, 1.0, 10.0], l1_ratio=[0.5, 0.7, 0.9], cv=5)
elasticnet_cv.fit(X_train_scaled, energia)
elasticnet_alpha = elasticnet_cv.alpha_
print("Alfa de ElasticNet:", elasticnet_alpha)
elasticnet_ratio = elasticnet_cv.l1_ratio_
elasticnet_score = elasticnet_cv.score(X_test_scaled, energia_test)
print("Score de EL en train:", elasticnet_cv.score(X_train_scaled, energia))
print("Score de EL en test:", elasticnet_score)

/home/robertbarac/maestria/seminario_en_ciencias/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/robertbarac/maestria/seminario_en_ciencias/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.738e-02, tolerance: 2.570e-02
  model = cd_fast.enet_coordinate_descent(


Alfa de Lasso: 0.1
Score de Lasso en training: 0.4982385749199857
Score de Lasso en test: 0.5332835197924009


/home/robertbarac/maestria/seminario_en_ciencias/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:1568: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/robertbarac/maestria/seminario_en_ciencias/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.622e-02, tolerance: 2.382e-02
  model = cd_fast.enet_coordinate_descent(
/home/robertbarac/maestria/seminario_en_ciencias/env/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regu

Alfa de ElasticNet: 0.1
Score de EL en train: 0.5630506170193579
Score de EL en test: 0.6029649539415813


### Tensil

In [71]:
# Crear un modelo de regresión Ridge con validación cruzada
ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1.0, 10.0, 30.0], cv=5)

# Entrenar el modelo con los datos de entrenamiento escalados
ridge_cv.fit(X_train_scaled, tensil)

# Obtener el valor óptimo de alpha seleccionado
best_alpha = ridge_cv.alpha_
print("Mejor valor de alpha:", best_alpha)

# Evaluar el modelo en el conjunto de prueba escalado
score = ridge_cv.score(X_test_scaled, tensil_test)
print("Puntaje en el conjunto de entrenamiento:", ridge_cv.score(X_train_scaled, tensil))
print("Puntaje en el conjunto de prueba:", score)

Mejor valor de alpha: 1.0
Puntaje en el conjunto de entrenamiento: 0.9658040171917774
Puntaje en el conjunto de prueba: 0.9721528387482337


In [75]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Crear una instancia del modelo de regresión Ridge
ridge = Ridge(alpha=1.0)

# Entrenar el modelo de regresión Ridge
ridge.fit(X_train_scaled, tensil)

# Realizar predicciones en el conjunto de prueba
y_pred = ridge.predict(X_test_scaled)

# Calcular el coeficiente de determinación (R-cuadrado) en el conjunto de prueba
r2 = r2_score(tensil_test, y_pred)

# Imprimir los resultados
print("Puntaje en el conjunto de entrenamiento:", ridge.score(X_train_scaled, tensil))
print("Puntaje en el conjunto de prueba:", r2)


Puntaje en el conjunto de entrenamiento: 0.9658040171917774
Puntaje en el conjunto de prueba: 0.9721528387482337


### Elongación

In [76]:
# Crear un modelo de regresión Ridge con validación cruzada
ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1.0, 10.0, 30.0], cv=5)

# Entrenar el modelo con los datos de entrenamiento escalados
ridge_cv.fit(X_train_scaled, elongacion)

# Obtener el valor óptimo de alpha seleccionado
best_alpha = ridge_cv.alpha_
print("Mejor valor de alpha:", best_alpha)

# Evaluar el modelo en el conjunto de prueba escalado
score = ridge_cv.score(X_test_scaled, elongacion_test)
print("Puntaje en el conjunto de entrenamiento:", ridge_cv.score(X_train_scaled, elongacion))
print("Puntaje en el conjunto de prueba:", score)

Mejor valor de alpha: 1.0
Puntaje en el conjunto de entrenamiento: 0.9426637278431481
Puntaje en el conjunto de prueba: 0.9465640801392357


### Secante

In [77]:
# Crear un modelo de regresión Ridge con validación cruzada
ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1.0, 10.0, 30.0], cv=5)

# Entrenar el modelo con los datos de entrenamiento escalados
ridge_cv.fit(X_train_scaled, secante)

# Obtener el valor óptimo de alpha seleccionado
best_alpha = ridge_cv.alpha_
print("Mejor valor de alpha:", best_alpha)

# Evaluar el modelo en el conjunto de prueba escalado
score = ridge_cv.score(X_test_scaled, secante_test)
print("Puntaje en el conjunto de entrenamiento:", ridge_cv.score(X_train_scaled, secante))
print("Puntaje en el conjunto de prueba:", score)

Mejor valor de alpha: 0.5
Puntaje en el conjunto de entrenamiento: 0.9575380926756124
Puntaje en el conjunto de prueba: 0.9589818189183984


### Flexión

In [84]:
# Crear un modelo de regresión Ridge con validación cruzada
ridge_cv = RidgeCV(alphas=[1.0, 2.0, 2.5, 2.8, 5.0, 10.0, 30.0], cv=5)

# Entrenar el modelo con los datos de entrenamiento escalados
ridge_cv.fit(X_train_scaled, flexion)

# Obtener el valor óptimo de alpha seleccionado
best_alpha = ridge_cv.alpha_
print("Mejor valor de alpha:", best_alpha)

# Evaluar el modelo en el conjunto de prueba escalado
score = ridge_cv.score(X_test_scaled, flexion_test)
print("Puntaje en el conjunto de entrenamiento:", ridge_cv.score(X_train_scaled, flexion))
print("Puntaje en el conjunto de prueba:", score)

Mejor valor de alpha: 2.5
Puntaje en el conjunto de entrenamiento: 0.6595680506523969
Puntaje en el conjunto de prueba: 0.30334145543000457


### Polinómica para Tracción

In [60]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Crear características polinómicas
degree = 2  # Grado del polinomio
poly_features = PolynomialFeatures(degree=degree, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train_scaled)

# Crear y entrenar el modelo de regresión polinómica
poly_model = make_pipeline(poly_features, LinearRegression())
poly_model.fit(X_train_scaled, traccion)

# Evaluar el modelo en el conjunto de prueba
X_test_poly = poly_features.transform(X_test_scaled)
score = poly_model.score(X_test_scaled, traccion_test)
print("Puntaje en training:", poly_model.score(X_train_scaled, traccion))
print("Puntaje en el conjunto de prueba:", score)


Puntaje en training: 1.0
Puntaje en el conjunto de prueba: 0.2119510654830119


In [65]:
# Crear características polinómicas
degree = 2  # Grado del polinomio
poly_features = PolynomialFeatures(degree=degree, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train_scaled)

# Crear y entrenar el modelo de regresión polinómica
poly_model = make_pipeline(poly_features, LinearRegression())
poly_model.fit(X_train_scaled, energia)

# Evaluar el modelo en el conjunto de prueba
X_test_poly = poly_features.transform(X_test_scaled)
score = poly_model.score(X_test_scaled, energia_test)
print("Puntaje en training:", poly_model.score(X_train_scaled, energia))
print("Puntaje en el conjunto de prueba:", score)

Puntaje en training: 1.0
Puntaje en el conjunto de prueba: 0.5657345549346151
